In [6]:
from langgraph.graph import StateGraph,END
from langgraph.types import Command
from typing import TypedDict
from langgraph.types import interrupt
from langgraph.checkpoint.memory import MemorySaver


memory = MemorySaver()
class State(TypedDict):
    text:str


def nodeA(state:State):
    print("Node A")
    return Command(
        goto="node_b",
        update={
            "text":state["text"]+"a"
        }
    )


def nodeB(state:State):
    print("Node B")
    human_response = interrupt("Do You Want To Go To C or D? Type (C/D)")
    print("Human Values",human_response)
    if(human_response=="C"or human_response=="c"):
            return Command(
            goto="node_c",
            update={
            "text":state["text"]+"b"
            }
    )
    elif(human_response=="D"or human_response=="d"):
         return Command(
            goto="node_d",
            update={
            "text":state["text"]+"b"
            }
    )
         

    



def nodeC(state:State):
    print("Node C")
    return Command(
        goto="node_d",
        update={
            "text":state["text"]+"c"
        }
    )

def nodeD(state:State):
    print("Node D")
    return Command(
        goto=END,
        update={
            "text":state["text"]+"d"
        }
    )


# Dont have to add edge here we manage upwards
graph = StateGraph(State)
graph.add_node("node_a",nodeA)
graph.add_node("node_b",nodeB)
graph.add_node("node_c",nodeC)
graph.add_node("node_d",nodeD)

graph.set_entry_point("node_a")

app = graph.compile(checkpointer=memory)
config = {"configurable":{"thread_id":"1"}}
initial_state = {
     "text":""
}
response = app.invoke(initial_state,config=config,stream_mode="updates")
response

Node A
Node B


[{'node_a': {'text': 'a'}},
 {'__interrupt__': (Interrupt(value='Do You Want To Go To C or D? Type (C/D)', resumable=True, ns=['node_b:2e58408d-646f-deb0-0f3d-9f3c5c870e5e']),)}]

In [7]:
user_input = app.invoke(Command(resume="D"),config=config,stream_mode="updates")
user_input

Node B
Human Values D
Node D


[{'node_b': {'text': 'ab'}}, {'node_d': {'text': 'abd'}}]